# Skill-Server Usage

This notebook should quickly outline the basic skills and their interfaces, i.e. how to integrate them in the overall mess.

## Preliminaries

The skill-server is launched via the ``hrr_cobot_hw.launch`` file if the flag ``use_state_server`` is set to True (default).

The steps below can be run, if either the ``use_state_server``-flag has been set to False, or the node is killed via

```bash
rosnode kill /hrr_cobot/skill_server
```

### Test-mode 

For testing run

```bash
rosrun hrr_cobot_robot skill_server --no-compile-nb __ns:="hrr_cobot"
```

before executing the code within this notebook. For additional help and instructions run

```bash
rosrun hrr_cobot_robot skill_server --help
```

### Debugging mode

For debugging, use an IDE, e.g. [pycharm](https://www.jetbrains.com/pycharm/download/) or vscode(https://code.visualstudio.com/) with proper plugins, and run the ``skill_server`` script from ``hrr_cobot_robot->scripts->skill_server`` in debug mode, set breakpoints as needed and call the action-clients from here.

You may want to add the proper namespace to emulate identical beahvior, thus adjust the settings

![access-settings](./media/pycharm_skill_server_access_settings.png)

and add the namespace as a Parameter

![set namespace](./media/pycharm_skill_server.png)

## set up basics

For the code below, we assume that the rqt graph shows something like this

![rqt-graph](./media/skill_server_rqt_graph.png)

**NOTE: this graph is missing some actions and still uses an Arduino interface, but serves as a reference**

In [ ]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm

# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal

# set printing and plotting options
np.set_printoptions(precision=5, suppress=True)

from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

# replace this with current ROS_MASTER hostname
#hrr_common.utils.set_ros_environment(ros_master='hrrcobotLinux54')
hrr_common.set_ros_environment("129.187.147.188")

In [ ]:
rospy.init_node('skill_server_client')

## get ROS-action-server names

In [ ]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")

## action-client helper-functions

In [ ]:
def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client

In [ ]:
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()

## Start cobot

In [ ]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

In [ ]:
print(cobot)

In [ ]:
cobot.T_B_C_robot, cobot.T_B_E_robot

In [ ]:
cobot.T_B_E_robot.A, cobot.q

In [ ]:
rospy.sleep(5)
cobot.open_tool_changer()

In [ ]:
cobot.close_tool_changer()

### Check T_B_C transform

In [ ]:
cobot.init_sns_vel()

In [ ]:
cobot.stop()

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
v_test = np.zeros(6)
for t in range(600):
    v_test[5] = 3e-2
    cobot.update(u_cmd=v_test, u_cmd_frame="hrr_cobot.screwdriver_tip", sleep=True)
cobot.stop()

## Motion planner debugging and tests

In [ ]:
cobot._planner_interface.has_matlab

In [ ]:
data = np.load("screw_6.npy", allow_pickle=True)
data = data.item()
poses = data["pickup"]
poses

In [ ]:
data = np.load("screw_6.npy", allow_pickle=True)
data = data.item()
poses = data["pickup"]

In [ ]:
poses["poses_A"][0]

In [ ]:
T_B_E_calib = cobot.FK(cobot.q_calib)

In [ ]:
cobot.FK(np.r_[-1.5, cobot.q_calib[1:]])T_B_E_goali

In [ ]:
cobot.stochastic_move_to_pose(poses["poses_A"][1])

In [ ]:
cobot.stochastic_move_to_pose(cobot.FK(np.r_[1.5, cobot.q_calib[1:]])) #Stroke end axis 5 from shaftgrinder pose

In [ ]:
poses["poses_A"][1],poses["poses_A"][2]

In [ ]:
poses["poses_q"][0],poses["poses_q"][2]

In [ ]:
np.rad2deg([ 2.90515,  0.29346, -1.81037, -2.53444,  0.18333, -0.3138 ])

In [ ]:
get_IK_guess(poses["poses_A"][2], cobot.q_calib)

In [ ]:
get_IK_guess(poses["poses_A"][2],[ 2.90504,  0.28984, -1.80648, -2.49918,  0.6501 , -2.12023])

In [ ]:
cobot.stochastic_move_to_pose(poses["poses_A"][2])

In [ ]:
jt = cobot._planner_interface.get_joint_trajectory_to_joint_configuration(cobot.q_calib)

In [ ]:
np.rad2deg(jt)

In [ ]:
cobot.execute_joint_trajectory(joint_traj=jt, wait_for_feedback=True)

In [ ]:
if cobot._planner_interface.has_matlab: 
    joint_traj = cobot._planner_interface.plan_joint_trajectory(q_des)
    #joint_traj = cobot._planner_interface.get_joint_trajectory_to_joint_configuration(q_des)
    #If you want to use matlab IK:
    #joint_traj = self._planner_interface.get_joint_trajectory_to_pose(T_B_E_des)
cobot.execute_joint_trajectory(joint_traj=joint_traj, wait_for_feedback=True)
rospy.loginfo("re-/deactivate controllers")
cobot._joint_traj_handle.deactivate()
cobot.reactivate_controllers(cur_controllers)

In [ ]:
def get_IK_guess(T_B_E_des, current_q):
    qs_des = cobot.IK(T_B_E_des)
    #Get absolute distance to all q_des from current q
    qs_diff = [np.abs(elem-current_q) for elem in qs_des]
    #Find nearest q_des (sorted by joints)
    best_q_diff = qs_diff[0]
    best_idx = 0
    for k, q_diff in enumerate(qs_diff):
        if k==0:
            continue
        if q_diff[0] < best_q_diff[0]:
            best_q_diff = q_diff
            best_idx = k
        elif q_diff[0] == best_q_diff[0]:
            if q_diff[1] < best_q_diff[1]:
                best_q_diff = q_diff
                best_idx = k
            elif q_diff[1] == best_q_diff[1]:
                if q_diff[2] < best_q_diff[2]:
                    best_q_diff = q_diff
                    best_idx = k
                elif q_diff[2] == best_q_diff[2]:
                    if q_diff[3] < best_q_diff[3]:
                        best_q_diff = q_diff
                        best_idx = k
                    elif q_diff[3] == best_q_diff[3]:
                        if q_diff[4] < best_q_diff[4]:
                            best_q_diff = q_diff
                            best_idx = k
                        elif q_diff[4] == best_q_diff[4]:
                            if q_diff[5] < best_q_diff[5]:
                                best_q_diff = q_diff
                                best_idx = k
    return qs_des[best_idx]

In [ ]:
cobot.IK(cobot.T_B_E_robot)

In [ ]:
cobot._planner_interface.from_ros("/hrr_cobot")

In [ ]:
print(cobot._planner_interface._pub_joint_goal)

In [ ]:
Shaftgrinder_tf_1 = sm.SE3()
Shaftgrinder_tf_1.A[:4,:4] = [[0.9552, 0.02496, 0.2948, 0.4],
                              [-0.2956, 0.04037, 0.9545, 0],
                              [0.01192, -0.9989, 0.04594, 0.57],
                              [0,0,0,1]]

In [ ]:
Shaftgrinder_tf_1

In [ ]:
tempGoalPose = sm.SE3()
tempGoalPose.A[:4,:4] = [[ 0.98168,  0.06281,  0.17986,  0.33984],
        [ 0.0997 , -0.97386, -0.2041 , 0.1],
        [ 0.16234,  0.2183 , -0.96228,  0.60248],
        [ 0.     ,  0.     ,  0.     ,  1.     ]]

In [ ]:
tempGoalPose

In [ ]:
cobot.goTo(tempGoalPose)

In [ ]:
Shaftgrinder_tf_1, cobot.T_B_E_robot

In [ ]:
cobot.IK(Shaftgrinder_tf_1)

In [ ]:
cobot.IK(cobot.T_B_E_robot)

In [ ]:
cobot.FK(cobot.q_calib)

In [ ]:
cobot.IK(Shaftgrinder_tf_1)

In [ ]:
cobot.stochastic_move_to_pose(Shaftgrinder_tf_1)

In [ ]:
get_IK_guess(cobot.FK(cobot.q_calib), cobot.q), cobot.q_calib

In [ ]:
qs_des = cobot.IK(cobot.FK(cobot.q_calib))
#Get absolute distance to all q_des from current q
qs_diff = [np.abs(elem-cobot.q) for elem in qs_des]
#Find nearest q_des (sorted by joints)
best_q_diff = qs_diff[0]
best_idx = 0

In [ ]:
q_diff = qs_diff[1]

In [ ]:
q_diff[0] < best_q_diff[0]

In [ ]:
cobot.q

### Camera hack

import tf2_ros
import geometry_msgs.msg
broadcaster = tf2_ros.StaticTransformBroadcaster()
static_transformStamped = geometry_msgs.msg.TransformStamped()

static_transformStamped.header.stamp = rospy.Time.now()
static_transformStamped.header.frame_id = "hrr_cobot.base_link"
static_transformStamped.child_frame_id = "camera_link"
static_transformStamped.transform.translation.x = 0.7
static_transformStamped.transform.translation.y = 0.3
static_transformStamped.transform.translation.z = 0.02

static_transformStamped.transform.rotation.x = 0.0
static_transformStamped.transform.rotation.y = 0.0
static_transformStamped.transform.rotation.z = 0.0
static_transformStamped.transform.rotation.w = 1.0

broadcaster.sendTransform(static_transformStamped)

#Hide CObot
joi = cobot.q_calib
joi[0]=-np.deg2rad(90)
cobot.move_to_joint_pose(joi, stochastic=True)

## Tool-Changer Handles

In [ ]:
rospy.sleep(10)
cobot.open_tool_changer()

In [ ]:
cobot.close_tool_changer()

## Network Test

In [ ]:
cobot.stop()

In [ ]:
cobot.reset()

In [ ]:
c

In [ ]:
print(cobot)

In [ ]:
cobot.FT_F

In [ ]:
cobot.update()

In [ ]:
cobot.hz = 100

In [ ]:
print(cobot)

In [ ]:
cobot.close_tool_changer()

In [ ]:
cobot.T_B_E_robot.A, np.rad2deg(cobot.q)

In [ ]:
cobot.goTo(sm.SE3([0,-0.2,0]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)

In [ ]:
cobot.update()

In [ ]:
cobot.reset()

In [ ]:
cobot.hz = 50

In [ ]:
cobot.goTo(sm.SE3([0,0,-0.1]) @ cobot.T_B_E_robot, v_max = 0.1, check_reachable=False)
cobot.goTo(sm.SE3([0,0,0.1]) @ cobot.T_B_E_robot, v_max = 0.1, check_reachable=False)

In [ ]:
cobot.goTo(sm.SE3([0,0,-0.1]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)
cobot.goTo(sm.SE3([0,0,0.1]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)
cobot.goTo(sm.SE3([0,0,-0.1]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)
cobot.goTo(sm.SE3([0,0,0.1]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)
cobot.goTo(sm.SE3([0,0,-0.1]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)
cobot.goTo(sm.SE3([0,0,0.1]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)
cobot.goTo(sm.SE3([0,0,-0.1]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)
cobot.goTo(sm.SE3([0,0,0.1]) @ cobot.T_B_E_robot, v_max = 0.01, check_reachable=False)

In [ ]:
cobot.hz = 80

## Cutting

In [ ]:
Shaftgrinder_pose_q = np.array([-0.08106, -0.23017, -2.33784, -0.93423, -0.8107 ,  2.32261])


In [ ]:
cobot.tool_id

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cobot.update()

In [ ]:
cobot.T_B_E_robot.t, cobot.T_B_C_robot.t

In [ ]:
Shaftgrinder_pose = sm.SE3()
Shaftgrinder_pose.A[:4,:4] = np.array([[-0.515, 0.02106, 0.8569, 0.3903],
                              [-0.8571, 0.0004331, -0.5151, -0.01507],
                              [-0.01122, -0.9998, 0.01782, 0.5722],
                              [0, 0, 0, 1]])

In [ ]:
Shaftgrinder_poseEL = sm.SE3()
Shaftgrinder_poseEL.A[:4,:4] = np.array([[   1     ,   -4.896e-12 , 0    ,     0.4004    ],
                                        [0    ,     0  ,       1  ,       0.1847  ],  
                                        [-4.896e-12 ,-1  ,       0 ,        0.2945 ],   
                                        [ 0   ,      0 ,        0 ,        1  ]]) 

In [ ]:
cobot.move_to_joint_pose(Shaftgrinder_pose_q)

In [ ]:
cobot.goTo(Shaftgrinder_poseEL)

In [ ]:
disp_A = []
disp_q = []
disp_open_tc = []
disp_joint_ctrl = []

In [ ]:
cobot.update_tf()
disp_A.append(cobot.T_B_E_robot)
disp_q.append(cobot.q)
disp_open_tc.append(0) #Set to 1 if the current pose is the one where tool changer is opened
disp_joint_ctrl.append(0) #Set to 1 if the current pose was reached by joint control
disp_A

In [ ]:
x = {"from_shaftgrindpose_toEL":
        {"poses_A": disp_A, "poses_q": disp_q,
        "open_tc": disp_open_tc, "use_joint_ctrl": disp_joint_ctrl}
    }
np.save("shaftgrind_help", x, allow_pickle=True)

In [ ]:
#WERTE WIE IN ECORESET. DONT TOUCH!!
material= MaterialType()
material = material.PLASTIC

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.516543880922,0.151079536263,0.2], "pointA")#[0.6921,0.2472,0.], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.516755472499,0.192494410614,0.2], "pointB")
## replace by vision data

In [ ]:
#WERTE ZUM VERÄNDERN
material= MaterialType()
material = material.PLASTIC

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.44808836914,0.35,0.1], "pointA")#[0.6921,0.2472,0.], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.44808836914,0.36,0.1], "pointB")
## replace by vision data

In [ ]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")
cobot.change_tool("shaftgrinder")

In [ ]:
Shaftgrinder_pose = sm.SE3()
Shaftgrinder_pose.A[:4,:4] = np.array([[-0.515, 0.02106, 0.8569, 0.3903],
                              [-0.8571, 0.0004331, -0.5151, -0.01507],
                              [-0.01122, -0.9998, 0.01782, 0.5722],
                              [0, 0, 0, 1]])

In [ ]:
!rosparam get /hrr_cobot/tool_name

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cobot.update()

In [ ]:
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)
#shaftgrinder_client.wait_for_server()
print('found')

In [ ]:
goal=CuttingGoal()
goal.surface_normal_start=normalA
goal.surface_normal_end=normalB
goal.start_location=pointA
goal.end_location=pointB
goal.material.material_type=material
goal.timeout=5000
goal.thickness = 0.005 #not important. can be anything

In [ ]:
print(goal)

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
shaftgrinder_client.wait_for_result()
shaftgrinder_client.get_result()

In [ ]:
shaftgrinder_client.cancel_goal()

In [ ]:
cobot.tool_controller.run_shaftgrinder(0.1, "slow")

In [ ]:
print(cobot)

In [ ]:
cobot.hz = 20

## Manual cutting to test

In [ ]:
cobot.tool

In [ ]:
cobot.T_B_E_robot@cobot.T_E_C_robot, cobot.T_B_C_robot

In [ ]:
cobot.T_B_C_robot.t-cobot.T_B_E_robot.t

In [ ]:
cobot.T_E_C_robot.t

In [ ]:
R_des = np.array([[0.9552, 0.02496, 0.2948],[-0.2956, 0.04037, 0.9545],[0.01192, -0.9989, 0.04594]])

Shaftgrinder_tf_1 = np.array([[0.9552, 0.02496, 0.2948, 0],
                              [-0.2956, 0.04037, 0.9545, 0],
                              [0.01192, -0.9989, 0.04594, 0],
                              [0, 0, 0, 1]])

CuttingPoseA = Shaftgrinder_tf_1
tip_falsch = Shaftgrinder_tf_1@cobot.T_E_C_robot
noetige_aenderung = hrr_cm.vec32np(goal.start_location.vector) - tip_falsch.t
[CuttingPoseA.t[0], CuttingPoseA.t[1], CuttingPoseA.t[2]] = CuttingPoseA.t + noetige_aenderung


CuttingPoseA = sm.SE3(hrr_cm.vec32np(goal.start_location.vector) + cobot.T_B_E_robot.t-cobot.T_B_C_robot.t)
CuttingPoseA.A[0:3, 0:3] = R_des

PrePoseA = sm.SE3([0, 0, 0.05]) @ CuttingPoseA
            
CuttingPoseB = sm.SE3(hrr_cm.vec32np(goal.end_location.vector) + cobot.T_B_E_robot.t-cobot.T_B_C_robot.t)
CuttingPoseB.A[0:3, 0:3] = R_des

PrePoseB = sm.SE3([0, 0, 0.05]) @ CuttingPoseB

In [ ]:
            CuttingPoseA = sm.SE3()
            CuttingPoseA.A[:4, :4] = Shaftgrinder_tf_1.A.copy()
            tooltip_offset_A = hrr_cm.vec32np(start_location.vector) - self.cobot.T_B_C_robot.t
            [CuttingPoseA.t[0], CuttingPoseA.t[1], CuttingPoseA.t[2]] = CuttingPoseA.t + tooltip_offset_A

## Usage examples

### calibration service

We start with the calibration action-server and send a calibration message as shown below.

**NOTE: ``calibration_file`` and ``data_file`` are optional, but if they are kept empty, you cannot plot the final results later**



In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
cobot.tool, cobot.T_B_E_robot.t - cobot.T_B_C_robot.t

In [ ]:
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)

In [ ]:
goal = CalibrateCobotGoal()
data_path = pathlib.Path.home() / "Documents" / "skill_recordings"
data_path.mkdir(parents=True, exist_ok=True)
tool_name = hrr_common.ros_utils.get_param(f'{cobot_ns}tool_name')
if tool_name == '':
    tool_name = 'nothing'
goal.calibration_file = str(data_path / f"{tool_name}.pkl")
goal.data_file = str(data_path / f"{tool_name}_calibration.npy")
goal.recalibrate = True
goal.scale_q4 = 0.5
goal.scale_q5 = 0.2
goal.scale_q6 = 0.5
goal

**WARNING: in case the tool-name above differs from the actual mounted one, please run the CORRECT line below**

```bash
rosparam set /hrr_cobot/tool_name "nothing"
rosparam set /hrr_cobot/tool_name "wsg_50_dsa"
rosparam set /hrr_cobot/tool_name "screwdriver"
rosparam set /hrr_cobot/tool_name "shaftgrinder"
rosparam set /hrr_cobot/tool_name "vacuum"
```

In [ ]:
calibrate_client.send_goal(goal)

In [ ]:
cancel(calibrate_client)

In [ ]:
print(cobot)

In [ ]:
cobot.move_to_joint_pose(cobot.q_calib)

In [ ]:
cobot.move_to_joint_pose(np.r_[cobot.q[0],  0.    , -1.5708,  0.    ,  1.5708,  0.  ])

#### inspect results

The results below are exemplarily shown by either calling the *result*-feedback or using the dedicated plotting function from an external terminal

**NOTE: plotting via shell-escape usually does not work**

The plotting should return a result similar to the one below:

![calibration example](./media/calibration_example.png)

In [ ]:
# get client result / feedback
get_result(calibrate_client)

In [ ]:
# plot results -> copy line to terminal
print(f"please run the following line in a separate terminal:\nrosrun hrr_cobot_robot plot_ft_calibration -p {goal.calibration_file} {goal.data_file}")

In [ ]:
#cobot.FT._calib_data.B_grav_vec #ziehe diese Werte ab (base frame)
cobot.FT._calib_data.bias #ziehe diese Werte ab (ft frame)

In [ ]:
f"rosrun hrr_cobot_robot plot_ft_calibration -p {goal.calibration_file} {goal.data_file}"

In [ ]:
cobot.update()

In [ ]:
cobot.B_F_msr

In [ ]:
cobot.FT_F

In [ ]:
cobot.stochastic_move_to_pose(cobot.FK(cobot.q_calib))

In [ ]:
cobot.FT._calib_data.B_grav_vec

In [ ]:
cobot.FT._calib_data.bias

In [ ]:
print(cobot)

In [ ]:
cobot.stop()

## Unscrewing 

For the unscrewing skill, there are additional inspections available:

- within pycharm you may run ``hrr_cobot_robot->scripts->devel->unscrew_devel`` in debug mode.
- from a notebook you may run ``notebooks->Unscrewing_tester``

Below, a short outline on how to use the skill-server is presented

In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
cobot.T_B_E_robot.A

In [ ]:
cobot.T_B_C_robot.t

In [ ]:
unscrewing_client = get_client(unscrew_action_srv_name, UnscrewAction)

In [ ]:
B_normal = sm.base.rotx(-5, "deg") @ sm.base.unitvec(np.r_[0., 0.0, 1.0])
B_screw_guess = np.r_[0.5789+0.013, 0.24-0.008, 0.30]#cobot.T_B_C_robot.t #np.r_[0.5, 0.25765, 0.26] #H #

In [ ]:
sc_goal = UnscrewGoal()
sc_goal.screw_location.header.frame_id = "hrr_cobot.base_link"
sc_goal.surface_normal.header.frame_id = "hrr_cobot.base_link"
sc_goal.surface_normal.vector = hrr_common.ros_utils.np2vec3(B_normal)
sc_goal.screw_location.point = hrr_common.ros_utils.np2vec3(B_screw_guess)
sc_goal.contact_force = 5
sc_goal.insertion_force = 10
sc_goal.timeout = 1200.0
# sc_goal

In [ ]:
unscrewing_client.send_goal(sc_goal)

In [ ]:
cobot.update()

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
cobot.stop()
cobot.reset()

In [ ]:
print(cobot)

In [ ]:
cobot.B_F_msr, np.linalg.norm(cobot.B_F_msr)

In [ ]:
forces = (sm.SE3(cobot.B_F_msr[0:3]) @ cobot.T_B_C).t
forces

In [ ]:
torques = (sm.SE3(cobot.B_F_msr[3:]) @ cobot.T_B_C).t
torques

### inspect results

In [ ]:
unscrewing_client.get_result()

In [ ]:
cancel(unscrewing_client)

## Vacuum Grasping

In [ ]:
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)

In [ ]:
obj_pos_guess = np.r_[0.4919, 0.1028, 0.3] #[0.66, 0.17, 0.07] is actual vision data for elamp cover
obj_orient_guess = np.quaternion(0,-0.6495, 0.029, 0.7597) #actual vision data (dont care for now)

release_pos = np.r_[0.5, 0.1, 0.35] # will drop the object here, careful!
release_orientation = np.quaternion(0,-0.715468876127, 0.0542776712358, 0.696533001155) #dont care for now

In [ ]:
vac_goal = VacuumGraspingGoal()
vac_goal.object_center.header.frame_id = "hrr_cobot.base_link"
vac_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
vac_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

vac_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
vac_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
vac_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

# vac_goal.timeout = 120.0
# vac_goal

In [ ]:
vacuum_client.send_goal(vac_goal)

In [ ]:
hrr_common.ros_utils.np2vec3(obj_pos_guess)cancel(vacuum_client)

In [ ]:
vacuum_client.get_result()

In [ ]:
cancel(vacuum_client)

In [ ]:
vacuum_client.cancel_all_goals()

## Pneumatic Finger

In [ ]:
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)

In [ ]:
obj_pos_guess = np.r_[0.5, -0.1, 0.33] #[0.712, 0.263, 0.0196] is actual vision data for battery
obj_orient_guess = np.quaternion(0.0,-0.4376,0.01394,0.8990) #actual vision data for battery
release_pos = np.r_[0.5, 0.1, 0.35] #choose so it makes sense
release_orientation = np.quaternion(0.0,-0.657320580033,-0.0241373269709,0.753224431702) #same orientaiton for ease

In [ ]:
pgrip_goal = FingerGraspGoal()
pgrip_goal.object_center.header.frame_id = "hrr_cobot.base_link"
pgrip_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
pgrip_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

pgrip_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
pgrip_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
pgrip_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

pgrip_goal.timeout = 10000.0
#pgrip_goal.gripper_open_width = 0.084
#pgrip_goal.gripper_close_width = 0
print(pgrip_goal)

In [ ]:
pgrip_client.send_goal(pgrip_goal)

In [ ]:
pgrip_client.get_result()

In [ ]:
cancel(pgrip_client)

## Tool Change routine

You may check the tool-change via

- rviz and checking the robot tool-chain to be correct
- varifying the ROS-parameter `/hrr_cobot/tool_name`

In [ ]:
!rosparam get /hrr_cobot/tool_name

In [ ]:
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)

In [ ]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")
cobot.change_tool("nothing")

In [ ]:
cobot.T_B_E_robot, cobot.q

In [ ]:
cobot.tool

In [ ]:
cobot.T_B_C_robot

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
cobot.close_tool_changer()

In [ ]:
und gen

In [ ]:
cobot.stop()

In [ ]:
print(cobot)

In [ ]:
tc_goal = ChangeToolGoal()
# set as needed
tc_goal.new_tool.type = tc_goal.new_tool.NONE
tc_goal.new_tool.type = tc_goal.new_tool.WSG_50
tc_goal.new_tool.type = tc_goal.new_tool.WSG_50_DSA
tc_goal.new_tool.type = tc_goal.new_tool.SHAFT_GRINDER
#tc_goal.new_tool.type = tc_goal.new_tool.SCREW_DRIVER
#tc_goal.new_tool.type = tc_goal.new_tool.VACUUM_GRIPPER

In [ ]:
tool_change_client.send_goal(tc_goal)

In [ ]:
np.save("testgoal",tc_goal)

In [ ]:
a = np.load("testgoal.npy",allow_pickle=True)

In [ ]:
a

In [ ]:
import yaml
from yaml.loader import SafeLoader

In [ ]:
cutting = yaml.load("cutting", Loader=SafeLoader)

In [ ]:
with open('cutting.yaml', 'r') as f:
    data = list(yaml.load_all(f, Loader=SafeLoader))
    print(data)

In [ ]:
stream = file('cutting.yaml', 'r')    # 'document.yaml' contains a single YAML document.
yaml.load(stream)

In [ ]:
print(cutting)

In [ ]:

cobot.FK(cobot.q_calib)

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
cancel(tool_change_client)

In [ ]:
cobot.T_B_E_robot

In [ ]:
cobot.T_B_E_robot

In [ ]:
cobot.q

In [ ]:
cobot.stop()

## Adaptive Grasping

This skill may be the most sensitive as the gripper driver is not really stable against using the tool-changer usage.

In (programming) theory, the gripper-driver is started via

```bash
roslaunch hrr_cobot_robot hrr_wsg_50.launch
```

via the WSG-watchdog, i.e. every time the tool is set to

```bash
rosparam set /hrr_cobot/tool_name "wsg_50_dsa"
```

and the gripper-IP can be pinged, but reality is different, so the rqt-graph below shows the skill-server with all nodes for the sensitive grasping

![rqt-graph](./media/hrr_wsg_setup_rqt.png)

**NOTE: The velocity based variant requires ```do-mpc```, so you need to run
```pip install do-mpc``` to replicate the setup below**

### Robot setup without skill-server

![rqt-graph](./media/rqt_without_skill_server.png)

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)

In [ ]:
print(cobot)

In [ ]:
grasp_goal = AdaptiveGraspingGoal()
T_B_C_test = sm.SE3(0.55, 0, 0.1)
grasp_goal.object_center.pose.position = hrr_common.np2vec3(T_B_C_test.t)
grasp_goal.object_center.pose.orientation = hrr_common.np2quat(hrr_common.rotmat2quat(T_B_C_test.R))
grasp_goal.contact_force = 5.0
grasp_goal.timeout = 2000.0
grasp_goal.dispose_off_location_pose.position = hrr_common.np2vec3(np.r_[0.45,-0.1,0.1])
grasp_goal

In [ ]:
cobot.T_E_C_robot @ cobot.T_B_C_robot

In [ ]:
grasping_client.send_goal(grasp_goal)

In [ ]:
grasping_client.get_result()

In [ ]:
cancel(grasping_client)

In [ ]:
np.linalg.norm(inverseboi[0]),np.linalg.norm(inverseboi[1]), np.linalg.norm(inverseboi[2]), np.linalg.norm(inverseboi[3])

In [ ]:
def find_best_IK(IKs):

    norm = []
    for i in range(len(IKs)):
        norm.append(np.linalg.norm(IKs[i]))

    bestIK = IKs[np.argmin(norm)]
    return bestIK


In [ ]:
np.rad2deg(np.r_[ 2.112,  0,    -1.571,  0.,     1.071,  0.   ])

In [ ]:
q = find_best_IK(goal)
q

In [ ]:
cobot.T_B_C_robot

In [ ]:
cobot.stop()

In [ ]:
print(cobot)

In [ ]:
cobot.move_to_joint_pose(xgoal, stochastic=True)

In [ ]:
cobot.stop()

In [ ]:
cobot.q

In [ ]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")


In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("wsg_50")